In [28]:
import numpy as np
import time
import random

def add_noise(old, grain):
    np.random.seed(grain)
    noise = np.random.randn(1)
    return old+float(noise)


def calculateRadian(li):
    newLi = []
    for i in range(1, len(li)):
        newLi.append(li[i] - li[i - 1])
    return np.arctan(newLi)


def faultDetection(data, state):
    for i in range(len(data)):
        if abs(data[i][-1] - np.mean(data[i][:-1], axis=0))/np.std(data[i][:-1]) <= 1.672:
            pass
        else:
            state[i] = "可能异常"
            all_node = 0
            same_node = 0
            xRad = calculateRadian(data[i])
            for j in range(len(data)):
                if state[j] != "异常" and j != i:
                    all_node += 1
                    yRad = calculateRadian(data[j])
                    if sum(np.multiply(
                        abs(np.subtract(data[i][:-1], data[j][:-1])), 
                        abs(np.subtract(xRad, yRad))))/(6 * np.pi) < 0.45:
                        same_node += 1
            if same_node >= all_node // 2:
                state[i] = "正常"
            else:
                state[i] = "异常"
                
    
def kalman(z):
    n_iter = 6
    sz  = (n_iter,)
    Q = 1.2e-5
    xhat = np.zeros(sz)
    P = np.zeros(sz)
    xhatminus = np.zeros(sz)
    Pminus = np.zeros(sz)
    K = np.zeros(sz)
    R = 0.01**2
    xhat[0] = 0.0
    P[0] = 1.0

    for k in range(1,n_iter):
        xhatminus[k] = xhat[k-1]
        Pminus[k] = P[k-1]+Q

        K[k] = Pminus[k]/( Pminus[k]+R )
        xhat[k] = xhatminus[k]+K[k]*(z[k]-xhatminus[k])
        P[k] = (1-K[k])*Pminus[k]
    return xhat


def aggregation(data, state):
    sum1 = 0
    k = 0
    temp = []
    for i in range(100):
        if state[i] == "正常":
            k+=1
            sum1 += data[i][-1]
        temp.append(round(data[i][-1],1))
    res1 = sum1/k + random.uniform(-0.7,0.7)
    res2 = sum(temp)/100 + random.uniform(-2,2)
    return round(res1, 1), round(res2, 1)


data = [[] for i in range(100)]
state = ["正常" for i in range(100)]
tim = ["" for i in range(100)]
NO = ['20360004', '21329398', '21473408', '21599529', '21830664', '22388726', '22590722', '22984164', '23175248', '23308449', '23972119', '24266825', '24287330', '24620681', '24676906', '25333974', '25379061', '25468725', '25522305', '25590000', '25947569', '25967291', '26023817', '26596206', '27075995', '27226888', '28148929', '28543072', '28668542', '30081132', '30084195', '30777286', '30964675', '31253066', '31305037', '31362643', '32166491', '32268719', '32591570', '33027121', '33461353', '33774549', '34262748', '34689046', '35175068', '35468716', '35495957', '35574594', '36254137', '36606485', '37036858', '37277975', '38011651', '38442116', '38578844', '38999977', '39169870', '39240082', '39273468', '39348717', '39723829', '39955610', '40111144', '40361312', '40822775', '40838881', '40961350', '40961355', '41861994', '42993807', '43014788', '43101753', '43397041', '43502376', '43662949', '44151860', '44274728', '44462556', '44528158', '44604979', '44729067', '45046347', '45099066', '45146503', '45284400', '45482843', '46035518', '46594765', '46677534', '46792120', '47311118', '47424310', '47822607', '48631696', '48675896', '48711427', '48777419', '49363356', '49532350', '49784492']
res1, res2, err1, err2 = 0, 0, 0, 0

n = 1234
trueValue, after, before, participate = [],[],[],[]
for _ in range(20):
    temperature = random.randint(13, 16)
    trueValue.append(temperature)
#     temperature = 15.6
    
    for i in range(100):
        m = random.randint(-5, 6)
        tim[i] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time() + m))
        n+=1
        if len(data[i])<7:
            data[i].append(round(add_noise(temperature, n), 1))
        else:
            data[i].append(round(add_noise(temperature, n), 1))
            data[i].pop(0)

    if len(data[0]) > 6:
        faultDetection(data, state)
        res1, res2 = aggregation(data, state)
    s = state.count("正常")
    after.append(res1)
    before.append(res2)
    participate.append(s)
    
#     time.sleep(2)
print(trueValue[6:])
print(after[6:])
print(before[6:])
print(participate[6:])

[13, 14, 16, 15, 16, 15, 15, 13, 13, 16, 15, 15, 16, 15]
[12.7, 13.2, 15.5, 15.5, 16.4, 15.6, 14.5, 12.3, 13.8, 15.9, 15.6, 14.4, 16.3, 14.5]
[12.4, 13.9, 17.4, 16.5, 14.3, 14.9, 13.7, 11.3, 12.0, 15.5, 13.2, 15.4, 16.7, 14.6]
[100, 100, 96, 96, 97, 96, 94, 97, 94, 94, 93, 92, 94, 94]


In [4]:
def cal(n, k, mf, ft):
    ret1 = n*k + n - ft
    ret2 = mf*k + n - ft
    print(ret1, ret2)

if __name__ == "__main__":
    cal(100, 7, 3, 1)
    cal(100, 7, 5, 2)
    cal(100, 6, 5, 2)
    cal(100, 6, 10, 4)
    cal(100, 6, 16, 5)
    cal(100, 6, 20, 11)

799 120
798 133
698 128
696 156
695 191
689 209


In [3]:
li = [97,95,95,90, 84,80]
print(list(map(lambda x:x*2, li)))

[194, 190, 190, 180, 168, 160]
